### Install modules:

In [0]:
pip install sentence-transformers

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install transformers

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install torch

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install gradio

### Call out libraries:

In [0]:
import transformers
from transformers import pipeline
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import numpy as np

from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

import gradio as gr

### Define functions:

In [0]:
def text_clean(text):
  try:
    text = text.lower()
    text = re.sub(r'https\S+', '', text) # remove urls starting with https
    text = re.sub(r'www\S+', '', text) # remove urls starting with www
    text = re.sub(r'\S*@\S*\s?', '', text)# remove email address
    text = re.sub(r"[^\w\s]", '',text) #remove puntucation
    text = re.sub(r"[0-9]", '',text) #remove numbers
    text = re.sub(r"_", "", text)
    # define a list of names
    pattern = r'\b(?:' + '|'.join(re.escape(name) for name in names) + r')\b'
    text = re.sub(pattern, "", text)
    text = re.sub(r"\s+", ' ',text) #remove multiple space
  except:
    pass
  return text

In [0]:
def func(question):
  result = nlp(question = question, context=context1)
  return result['answer']


### Read in Dataset

In [0]:
df = pd.read_csv("/dbfs/FileStore/Szhao/Xact_Data/hackthon/hurricane_data.csv",encoding = 'unicode_escape')

# QA Bot

In [0]:
##Select a smaller portion of the large dataset:
index_keep = np.where(df['Retweets']>5)[0]
print(len(index_keep))
df_short = df.iloc[index_keep]

18396


In [0]:
## Clean the text
df_short['Tweet_Clean'] = df_short.Tweet.apply(text_clean)

df_short = df_short.dropna(axis='rows',subset=['Tweet_Clean'])

<command-1791734583084974>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['Tweet_Clean'] = df_short.Tweet.apply(text_clean)


In [0]:
context  = df_short["Tweet_Clean"]
context1 = ", ".join(context)

#### Run the model on the dataset:

We have used the roBERTa model from Hugging Face. We have shown the first step in the process where we create Tokens.

In [0]:

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
# QA_input = {
#     'question': 'Why is model conversion important?',
#     'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
# }
# res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#Interface

In [0]:
# using context1 as input
  app = gr.Interface(fn=func, inputs = ['textbox'], outputs = 'textbox', title = 'Question Answering', description = 'Input question, then get answers.')

In [0]:
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://ba19f6d6780a32b122.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Out[5]: 

In [0]:
context1

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1791734583085123> in <cell line: 1>()
----> 1 context1

NameError: name 'context1' is not defined